# Based on the [Antibody Mimetics Workshop](https://colab.research.google.com/drive/187wIfCZTIbU7pgdrU1GDXxE_ihxRlmu3) by Michael Plevin & Jon Agirre

## Understanding this document in RStudio

You are using RStudio which is an integrated development environment (IDE) initially developed for R. It can now be used for many other languages including Python. You might find it useful because you have some familiar with with RStudio. It is also a very good way to create Quarto documents with code chunks in more than one language.

For example, this is an R code chunk. You can also run it by clicking on the green arrow in the top right of the chunk. Or do  `Cmd+Return` on a Mac or `Ctrl+Return` on Windows.



```{r}
getwd()
```

Notice:
-   prompt in the console is `>` as you are used to in R.
-   The Environment Pane is showing an R environment

This is a Python code chunk. You can also run it by clicking on the green arrow in the top right of the chunk. Or do  `Cmd+Return` on a Mac or `Ctrl+Return` on Windows.


In [ ]:
import os
os.getcwd()

Notice:
-   the R command `reticulate::repl_python()` was sent to the console to start Python
-   prompt in the console is `>>>`. This is the Python prompt. If you try typing R code like `getwd()` in the console at the `>>>` prompt you will get an error.
-   The Environment Pane is showing a Python environment containing an 'interface' to the r environment. This is the `r` object. 
-   If you type `r.getwd()` in the console at the Python prompt you don't get an error. You get the same result as you did in the R code chunk above


You can access R commands and R objects from you Python environment using the `r` object. For example, you can access the R vector object `an_r_obj` from Python using `r.an_r_obj`. 

Create a vector in R called `an_r_obj`:

```{r}
an_r_obj <- c(1, 2, 3)
str(an_r_obj)
```


Try to access it from Python:

In [ ]:
#| error: true
an_r_obj

You get an error because Python does not know about the R object `an_r_obj`. But you *can* access it using the `r` object:


In [ ]:
r.an_r_obj
type(r.an_r_obj)

You can 'copy' it to your python environment if you want:


In [ ]:
acopy = r.an_r_obj
type(acopy)

`acopy` is in your Python environment now:


Magic!

Datatypes differ between the languages. Reticulate converts some automatically - for example R vectors to Python lists. For some other datatypes you must explicitly convert. 


You can also access Python objects from R using the `py` object. For example, you can access the Python list object `a_python_obj` from R using `py$a_python_obj`. However you do need to have loaded the `reticulate` package in R first.


In [ ]:
a_python_obj = [4, 5, 6]

```{r}
library(reticulate)
```

```{r}
py$a_python_obj
```







## Getting started

This RStudio Project already has a folder to put the pbd file in. I have called it `pdb_files`. Jon called his `download`. When you are doing your own work you can make folders by
-   clicking on the New Folder button in the Files Pane
-   typing `mkdir -p pdb_files` in the *Terminal*
  

Make a folder for the pdb file. You could also do this "house-keeping" using the Files Pane in RStudio (New Folder)


  




Load the package


In [ ]:
import gemmi

Down load the pdb file. Again this command is not a python command but a shell command. You could just do `curl https://files.rcsb.org/download/4INS.pdb -o pdb_file/4INS.pdb` in the terminal.


In [ ]:
!curl https://files.rcsb.org/download/4INS.pdb -o pdb_file/4INS.pdb

I have used `curl` rather than `wget` because `wget` because most macOS and Windows do not come with `wget` by default. They mostly do have the very similar `curl`. Use is very similar but notice it is a lowercase `-o` for output rather than a capital `-O`. 

4INS is a 1.5 angstroems structure of insulin. We have seen structures of insulin in our first workshop. 

You can open the file to view it by clicking on it in the Files Pane.

Here are a few basic but interesting questions about a protein structure we can answer using Python programming:

-   What is the molecular weight of the protein?
-   What is the modelled sequence? We know the UniPro sequence, but is it the same as the one in the crystal?
-   What is the average B-factor per-residue? Would be neat to separate by main-chain and side-chain atoms.
-   Does the protein have any disulphide bonds?


Import the structure file:

In [ ]:
filename = "pdb_file/4INS.pdb" 
structure_file = gemmi.read_structure ( filename ) # remember gemmi.whatever?

In [ ]:
model = structure_file[0] # there can be more than one model per structure file!
                          # in computer science 0 is the first index, so we are
                          # getting the first (and only) model in the file

all the attributes and methods

In [ ]:
dir(model)

You can also click on it


In [ ]:
print ( "This protein weights", model.calculate_mass(), "Da" )

In [ ]:
print ( "This protein weights %.2f Da" % model.calculate_mass() )

In [ ]:
import numpy as np

In [ ]:
print(model[0])

In [ ]:
print(model[1])

In [ ]:
print(model[2])

In [ ]:
print(model[3])

In [ ]:
for polymer in model :
  print (polymer)

In [ ]:
for polymer in model :
  print (polymer)
  for steve in polymer :
    print(steve)

In [ ]:
for polymer in model :
  print (polymer)
  for steve in polymer :
    print(steve.name)

In [ ]:
for polymer in model :
  print (polymer)
  for steve in polymer :
    print(gemmi.find_tabulated_residue(steve.name).one_letter_code.upper())

In [ ]:
for polymer in model :
  sequence = ""
  print (polymer)
  for steve in polymer :
    sequence += gemmi.find_tabulated_residue(steve.name).one_letter_code.upper()
  print ("Polymer %s: %s" % (polymer.name,sequence))

In [ ]:
import statistics        # we will need to do stats, so we import the module

aminoacid_bfactors = [ ] # these are lists where we are going to store the
rest_bfactors      = [ ] # individual B-factors

for chain in model :
  for monomer in chain : # we'll call it monomer this time (name doesn't matter)
    for atom in monomer :
      if gemmi.find_tabulated_residue(monomer.name).is_amino_acid() :
        aminoacid_bfactors.append ( atom.b_iso )
      else :
        rest_bfactors.append ( atom.b_iso )

print ( "Protein part mean(stdev) : %.2f (%.2f)" % \
      ( statistics.mean ( aminoacid_bfactors ), statistics.stdev ( aminoacid_bfactors ) ))
print ( "Rest of atoms mean(stdev): %.2f (%.2f)" % \
      ( statistics.mean ( rest_bfactors ), statistics.stdev ( rest_bfactors ) ))